In [ ]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd


edoofy_base_url = "https://edoofa-portal.bubbleapps.io/api/1.1/obj"
edoofy_bearer_token = "2cde31d8f48919a2db1467cc06a56132"
edoofy_headers = {'Authorization': f'Bearer {edoofy_bearer_token}'}

ums_base_url = "https://app.edoofa.com/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}

async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    print(f"Starting to fetch data from {table} table...")
    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table} at cursor {cursor}: {response.status}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)

            print(f"Fetched {len(new_records)} records from {table} at cursor {cursor}")

            cursor += 100
            if len(new_records) < 100:
                print(f"Completed fetching data from {table}. Total records fetched: {len(records)}")
                break

    return pd.DataFrame(records)

async def fetch_engagement_data(session, base_url, headers, student_id, engagement_types):
    engagement_data = pd.DataFrame()
    tasks = []  # List to store tasks

    # Create a task for each engagement type
    for engagement_type in engagement_types:
        engagement_constraints = [
            {'key': 'student', 'constraint_type': 'equals', 'value': student_id},
            {'key': 'engagement-type', 'constraint_type': 'equals', 'value': engagement_type}
        ]
        task = fetch_table_data(session, base_url, headers, "Engagement", engagement_constraints)
        tasks.append(task)

    # Run tasks concurrently and wait for all to complete
    results = await asyncio.gather(*tasks, return_exceptions=True)

    # Process results
    for result in results:
        if isinstance(result, Exception):
            print(f"Request failed: {result}")
        else:
            engagement_data = pd.concat([engagement_data, result], ignore_index=True)

    return engagement_data

async def main():
    engagement_types = ['IE Call', 'IE Chat', 'Activity', 'Lesson']
    all_engagement_data = pd.DataFrame()
    error_records = []
    all_processed_data = []  # To store processed data for each EWYL group

    async with aiohttp.ClientSession() as session:
        # Fetch Attendance, Attendance Summary, Student, and Engagement Data
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")
        ums_avg_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance-Summary")
        ums_student_df = await fetch_table_data(session, ums_base_url, ums_headers, "Student")
        
        ewyl_group_names = ums_student_df['ewyl-group-name'].unique()

        for ewyl_group_name in ewyl_group_names:
            constraints = [{'key': 'EWYL-group-name', 'constraint_type': 'equals', 'value': ewyl_group_name}]
            edoofy_students = await fetch_table_data(session, edoofy_base_url, edoofy_headers, "Student", constraints=constraints)

            if edoofy_students.empty:
                error_message = f"No match found for EWYL-group-name: {ewyl_group_name}"
                print(error_message)
                error_records.append({'EWYL-group-name': ewyl_group_name, 'Error': 'No match found'})
                continue

            for student_id in edoofy_students['_id']:
                try:
                    student_engagement_data = await fetch_engagement_data(session, edoofy_base_url, edoofy_headers, student_id, engagement_types)
                    all_engagement_data = pd.concat([all_engagement_data, student_engagement_data], ignore_index=True)
                except Exception as e:
                    print(f"Failed to fetch engagement data for student {student_id}: {e}")
                    error_records.append({'_id': student_id, 'Error': str(e)})

            # Step 1: Process Engagement Data for each EWYL group
            processed_df = process_engagement_data(ums_att_df, all_engagement_data, edoofy_students, ewyl_group_name)
            all_processed_data.append(processed_df)

        # Step 2: Combine Processed Data into a single DataFrame
        combined_df = pd.concat(all_processed_data, ignore_index=True) if all_processed_data else pd.DataFrame()

        # Step 3: Process Attendance Summary using the combined DataFrame
        await process_att_summary(session, ums_base_url, ums_headers, combined_df, ums_avg_att_df)

    # Save errors and all engagement data to CSV
    errors_df = pd.DataFrame(error_records)
    errors_df.to_csv('unmatched_ewyl_group_names_errors.csv', index=False)
    all_engagement_data.to_csv('engagement_data.csv', index=False)

    print("Script completed.")

# Run the main function
await main()


